tracking position of eyes relative to each other!!! IE. HEIGHT.
put the blinking back in

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import sounddevice as sd
import threading
import time

############################################
# Configuration
############################################
BASE_FREQUENCY = 220.0       # Hz, baseline pitch
PITCH_RANGE = 600.0          # Hz added/subtracted based on eye direction
VOLUME_MIN, VOLUME_MAX = 0.05, 0.8
DISTORTION_MAX = 0.8
SAMPLE_RATE = 44100

############################################
# Shared state between camera + synth threads
############################################
pitch = BASE_FREQUENCY
volume = 0.3
distortion = 0.0
running = True

############################################
# Synthesizer thread
############################################
def synth_loop():
    global pitch, volume, distortion, running

    phase = 0.0
    block_size = 1024
    
    while running:
        t = np.arange(block_size) / SAMPLE_RATE
        freq = pitch

        # basic sine wave osc
        wave = np.sin(2 * np.pi * freq * t + phase)

        # distortion (tanh soft clip)
        if distortion > 0.01:
            wave = np.tanh(wave * (1 + distortion * 10))

        wave *= volume

        phase += 2 * np.pi * freq * (block_size / SAMPLE_RATE)

        sd.play(wave.astype(np.float32), SAMPLE_RATE, blocking=True)

    sd.stop()

############################################
# Eye tracking + mapping to synth parameters
############################################
mp_face = mp.solutions.face_mesh.FaceMesh(
    refine_landmarks=True,
    max_num_faces=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

def map_value(x, a, b, c, d):
    """Linear mapping from range [a,b] to [c,d]."""
    return (x - a) / (b - a) * (d - c) + c

cap = cv2.VideoCapture(0)

# Start synth thread
threading.Thread(target=synth_loop, daemon=True).start()

############################################
# Main webcam loop
############################################
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = mp_face.process(rgb)

        h, w, _ = frame.shape

        if result.multi_face_landmarks:
            face = result.multi_face_landmarks[0]

            # Eye-related points
            left_center = face.landmark[468]      # iris center (L)
            nose_base = face.landmark[1]          # stable ref point

            # ----- PITCH: horizontal eye position -----
            eye_x = left_center.x
            pitch = BASE_FREQUENCY + map_value(
                eye_x,
                0.3, 0.7,          # typical range center-left to center-right
                -PITCH_RANGE, PITCH_RANGE
            )

            # ----- VOLUME: distance to camera -----
            dz = abs(left_center.z - nose_base.z)
            volume = np.clip(
                map_value(dz, 0.015, 0.09, VOLUME_MAX, VOLUME_MIN),
                VOLUME_MIN, VOLUME_MAX
            )

            # ----- DISTORTION: eye openness -----
            top_lid = face.landmark[386]
            bottom_lid = face.landmark[374]
            openness = abs(top_lid.y - bottom_lid.y)

            distortion = np.clip(
                map_value(openness, 0.01, 0.06, 0.0, DISTORTION_MAX),
                0.0, DISTORTION_MAX
            )

        # Display data on screen
        cv2.putText(frame, f"Pitch: {pitch:.1f} Hz", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Volume: {volume:.2f}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Distortion: {distortion:.2f}", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        cv2.imshow("Eye Synth", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break

finally:
    running = False
    cap.release()
    cv2.destroyAllWindows()


I0000 00:00:1764726060.387044 6417924 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1764726060.388469 6418019 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764726060.393549 6418019 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764726061.235861 6418024 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
